## **SLE GWAS: Phenotype Derivation & Sample QC**

This notebook derives systemic lupus erythematosus (SLE) case/control phenotypes from UK Biobank ICD-10 diagnosis codes and performs sample quality control for genome-wide association analysis.

**Analysis Environment:**
- Platform: UK Biobank Research Analysis Platform (RAP)
- Instance: Spark cluster, mem1_hdd1_v2_x4, 2 nodes
- Data: UK Biobank phenotype dataset

### **Step 1: Extract Dataset Metadata**

In [ ]:
import dxpy # DNAnexus Python SDK for interaction with RAP
import subprocess

# Automatically locate dispensed dataset
dispensed_dataset_id = dxpy.find_one_data_object(
    typename='Dataset',
    name='app*.dataset', 
    folder='/', 
    name_mode='glob'
)['id']

print("Dataset ID:",dispensed_dataset_id)

# Get project ID & create full dataset reference
project_id = dxpy.find_one_project()["id"]
print("Project ID:",project_id)

dataset = (':').join([project_id, dispensed_dataset_id])
print("Full dataset reference:", dataset)

# Extract dataset metadata
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

**Output:** Three CSV files containing dataset metadata:
- `data_dictionary.csv` - Field definitions and metadata
- `codings.csv` - Lookup tables for medical codes (e.g., ICD-10)
- `entity_dictionary.csv` - Entity relationships

In [ ]:
# Inspect data_dictionary.csv
import glob
import os
import pandas as pd

path = os.getcwd()
data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0] # Find first file matching pattern *.data_dictionary.csv in current directory
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

### **Step 2: Define Required Data Fields**

Extract data fields needed for phenotype definition and sample QC.

In [ ]:
from distutils.version import LooseVersion

def field_names_for_ids(field_id):
    """Extract all field names matching given field IDs."""
    field_names = ["eid"]
    for _id in field_id:
        pattern = r'^p{}(_i\d+)?(_a\d+)?$'.format(_id)
        matches = data_dict_df[data_dict_df.name.str.match(pattern)].name.values
        field_names += list(matches)
    
    field_names = sorted(field_names, key=lambda n: LooseVersion(n)) # Sort field names in intuitive order
    field_names = [f"participant.{f}" for f in field_names]
    return ",".join(field_names)

field_ids = [
    '31',     # Sex
    '21022',  # Age at recruitment
    '22001',  # Genetic sex
    '22006',  # Genetic ethnic grouping
    '22009',  # Genetic principal components (1-40)
    '22019',  # Sex chromosome aneuploidy
    '22021',  # Genetic kinship
    '22027',  # Heterozygosity/missing rate outliers
    '41270'   # ICD-10 diagnoses (primary & secondary)
]

field_names = field_names_for_ids(field_ids)
print(f"Extracted {len(field_names.split(','))} fields")

In [ ]:
print(field_names)

**Field naming convention:** `p<field_id>_iX_aY`
- `i` (instance): Data collection timepoint
- `a` (array): Multiple values per instance (e.g., multiple diagnoses)

### **Step 3: Extract Phenotype Data**

In [ ]:
# Generate SQL query for field extraction
cmd = ["dx", "extract_dataset", dataset, "--fields", field_names, 
       "--delimiter", ",", "--output", "extracted_data.sql", "--sql"]
subprocess.check_call(cmd)

import pyspark

sc = pyspark.SparkContext() # Initialize Spark context
spark = pyspark.sql.SparkSession(sc) # Initialize Spark SQL session

# Execute SQL query via Spark
with open("extracted_data.sql", "r") as file: 
    retrieve_sql=""
    for line in file: 
        retrieve_sql += line.strip() # Join into single SQL query string, strips whitespaces

temp_df = spark.sql(retrieve_sql.strip(";")) # Run SQL query using Spark, strip trailing ; and return as Spark dataframe
pdf = temp_df.toPandas()

print(f"Extracted data: {pdf.shape[0]:,} samples, {pdf.shape[1]} fields")
pdf.head()

**Reformat column headers for readability**

In [ ]:
import re

pdf = pdf.rename(columns=lambda x: re.sub('participant.','',x)) # Drop 'participant.' prefix with regex
pdf.head()

### **Step 4: Define SLE Phenotype**

#### **Step 4.1: Extract SLE ICD-10 Codes from `codings_csv`**

In [ ]:
codings_csv = glob.glob(os.path.join(path, "*.codings.csv"))[0]
codings_df = pd.read_csv(codings_csv)
codings_df.head()

# Extract M32* codes (systemic lupus erythematosus)
sle_icd10_codes = list(
    codings_df[
        (codings_df["coding_name"] == "data_coding_19") & 
        (codings_df["parent_code"] == "M32")
    ]["code"]
)

print(f"SLE ICD-10 codes (n={len(sle_icd10_codes)}): {sle_icd10_codes}")

**Expected output:** Should be similar to the SLE ICD-10 codes `['M32.0', 'M32.1', 'M32.8', 'M32.9']` listed on the UKB Showcase.

#### **Step 4.2: Classify SLE Cases and Controls**

**Case definition:** Presence of ≥1 SLE ICD-10 code in p41270 (includes both primary & secondary diagnoses) <br>
**Control definition:** Absence of SLE ICD-10 codes in p41270

In [ ]:
import ast
import numpy as np

# Replace NaN with string None for p41270
pdf["p41270"] = pdf["p41270"].replace(np.nan, "None")

# Extract each participant's ICD10 codes and append as new column
def extract_icd10_codes(row):
    icd10_codes = row['p41270'] or []
    return list(set(icd10_codes))

pdf['icd10_codes'] = pdf.apply(extract_icd10_codes, axis=1)

def classify_sle(row):
    """Return 1 if participant has any SLE code, else 0."""
    return 0 if set(row['icd10_codes']).isdisjoint(sle_icd10_codes) else 1

# Determine each sample's SLE status and append as new column
pdf['has_sle_icd10'] = pdf.apply(classify_sle, axis=1)

In [ ]:
pdf.head()

In [ ]:
counts = pdf['has_sle_icd10'].value_counts()
print("\n=== Pre-QC SLE Distribution ===")
print(f"Cases:      {counts.get(1, 0):,}")
print(f"Controls:   {counts.get(0, 0):,}")
print(f"Total:      {counts.sum():,}") 
print(f"Prevalence: {counts.get(1, 0)/counts.sum()*100:.2f}%")

### **Step 5: Sample Quality Control**

**Filters applied:**
1. Sex concordance (reported sex == genetic sex)
2. White British ancestry (to reduce population stratification)
3. No sex chromosome aneuploidy
4. No excess kinship (≤10 third-degree relatives)
5. No heterozygosity/missingness outliers

**Note:** Ancestry restriction limits generalizability but improves statistical power by reducing stratification.

In [ ]:
# Remove participants with missing key covariates
p22009_cols = [col for col in pdf.columns if col.startswith("p22009")] # genetic principal components 
key_covariates = ['p31',    # sex
                  'p21022', # age at recruitment
                  'p22006', # genetic ethnic grouping
                  'p22001'  # genetic sex
                 ] + p22009_cols 

pdf = pdf.dropna(subset=key_covariates)

print(f"After removing missing key covariates: {pdf.shape[0]:,}")

# Apply Sample QC filters
pdf_qced = pdf[
    (pdf['p31'] == pdf['p22001']) &      # Sex concordance
    (pdf['p22006'] == 1) &               # White British
    (pdf['p22019'].isnull()) &           # No aneuploidy
    (pdf['p22021'] != 10) &              # No excess relatives
    (pdf['p22027'].isnull())             # No het/missing outliers
]

print(f"After Sample QC filters: {pdf_qced.shape[0]:,}")

In [ ]:
qc_counts = pdf_qced['has_sle_icd10'].value_counts()
print("\n=== Post-QC SLE Distribution ===")
print(f"Cases:      {qc_counts.get(1, 0):,}")
print(f"Controls:   {qc_counts.get(0, 0):,}")
print(f"Total:      {qc_counts.sum():,}") 
print(f"Prevalence: {qc_counts.get(1, 0)/qc_counts.sum()*100:.2f}%")

### **Step 6: Format for REGENIE Input**

In [ ]:
# Rename columns for clarity
pdf_qced = pdf_qced.rename(columns=lambda x: re.sub('p22009_a', 'pc', x)) # Rename genetic principal components to pc1...pc40
pdf_qced = pdf_qced.rename(columns={
    'eid': 'IID',
    'p31': 'sex',
    'p21022': 'age',
    'p22001': 'genetic_sex',
    'p22006': 'ethnic_group',
    'p22019': 'sex_chromosome_aneuploidy', 
    'p22021': 'kinship_to_other_participants', 
    'p22027': 'outliers_for_heterozygosity_or_missing',
    'p41270': 'all_icd10_codes'
})

# Format ICD-10 codes as comma-separated string
pdf_qced['icd10_codes'] = pdf_qced['icd10_codes'].apply(
    lambda x: 'NA' if len(x) == 0 else ",".join(x)
)

# Add FID column (REGENIE input requirement)
pdf_qced['FID'] = pdf_qced['IID']
pdf_qced = pdf_qced[['FID', 'IID'] + [col for col in pdf_qced.columns if col not in ['FID', 'IID']]]

In [ ]:
pdf_qced

### **Step 7: Determine age and sex distributions of samples**

In [ ]:
cases = pdf_qced[pdf_qced['has_sle_icd10'] == 1]
controls = pdf_qced[pdf_qced['has_sle_icd10'] == 0]

In [ ]:
# Create a summary table for sex distribution
print("\n=== AGE SUMMARY TABLE ===")
sex_comparison = pd.DataFrame({
    'Cases': [
        len(cases),
        f"{(cases['sex']==0).sum()} ({(cases['sex']==0).sum()/len(cases)*100:.1f}%)",
        f"{(cases['sex']==1).sum()} ({(cases['sex']==1).sum()/len(cases)*100:.1f}%)"
    ],
    'Controls': [
        len(controls),
        f"{(controls['sex']==0).sum()} ({(controls['sex']==0).sum()/len(controls)*100:.1f}%)",
        f"{(controls['sex']==1).sum()} ({(controls['sex']==1).sum()/len(controls)*100:.1f}%)"
    ]
}, index=['N', 'Female', 'Male'])

print(sex_comparison)

In [ ]:
# Create a summary table for age distribution
print("\n=== AGE BY DECADE: CASES vs CONTROLS ===")

# Define age bins
age_bins = [40, 50, 60, 70]
age_labels = ['40-49', '50-59', '60-69']

# Cases
cases_age_groups = pd.cut(cases['age'], bins=age_bins, labels=age_labels, right=False)
cases_age_dist = cases_age_groups.value_counts().sort_index()

# Controls
controls_age_groups = pd.cut(controls['age'], bins=age_bins, labels=age_labels, right=False)
controls_age_dist = controls_age_groups.value_counts().sort_index()

# Create comparison table
age_comparison = pd.DataFrame({
    'Cases_N': cases_age_dist,
    'Cases_%': (cases_age_dist / len(cases) * 100).round(1),
    'Controls_N': controls_age_dist,
    'Controls_%': (controls_age_dist / len(controls) * 100).round(1)
})

print(age_comparison)

### **Step 8: Save Phenotype File to Project**

In [ ]:
# Save phenotype table as csv files
pdf_qced.to_csv('sle_pqc_gwas.phe', sep='\t', na_rep='NA', index=False, quoting=3)
pdf_qced.to_csv('sle_pqc_gwas.txt', sep='\t', na_rep='NA', index=False, quoting=3)

In [ ]:
%%bash 
# Upload the phenotype file back to the RAP project 
dx upload sle_pqc_gwas.phe -p --path /02.Phenotype_SampleQC/
dx upload sle_pqc_gwas.txt -p --path /02.Phenotype_SampleQC/